**实验目标：**

通过本实验，你将深入了解和实践说话人识别技术，并掌握利用声音特征进行有效说话人识别的基本方法，了解不同特征和模型对识别准确率的影响。

实验的核心目标是使用TIMIT数据集来训练一个说话人识别系统，涵盖数据预处理、特征提取、模型训练和评估等关键步骤。


**实验方法：**

**1. 数据预处理和划分(可选)：**
  - 数据集下载地址（4月17日前有效）：https://f.ws59.cn/f/du8yd2536vl
  - 为了方便大家，我们提供了划分好的TIMIT数据集结构，当然你也可以根据需求自行划分该数据集。
  - 为简化难度，我们排除了SA的两个方言句子，并在剩余的8个句子中选取了SX的5个句子和SI的1个句子作为训练集，SI的另外2个句子作为测试集。
  - 该链接下载的数据集只保留了音频文件，完整数据集（包含音频对应文本、标注等信息）可参见备注链接下载。
  
**2. 特征提取：**
  - 学习并实现包括但不限于MFCC特征等特征的提取，探索声音信号的频率和时间特性。
  - 鼓励尝试和比较其他特征提取方法，例如LPCC或声谱图特征，以理解不同特征对识别性能的影响。
  
**3. 模型选择和训练：**
  - 探索并选择适合的分类器和模型进行说话人识别，如GMM、Softmax分类器或深度学习模型。
  - 实现模型训练流程，使用训练集数据训练模型。
  
**4. 评估和分析：**
  - 使用准确率作为主要的评价指标在测试集上评估模型性能。
  - 对比不同特征和模型的性能，分析其对说话人识别准确率的影响。
  - 可视化不同模型的识别结果和错误率，讨论可能的改进方法。

**实验要求：**
  - 1.选择并实现至少一种特征的提取，并鼓励尝试其他特征提取方法。
  - 2.选择并实现至少一种分类器或模型进行说话人识别，并使用准确率评估指标评估其性能。
  - 3.通过实验对比、分析和可视化，撰写详细的实验报告，包括实验目的、实验方法、结果分析和结论。
  - 4.实验报告应以清晰、逻辑性强的形式呈现，图表和结果应清楚明了。

**其他说明：**
  - 实验的最终打分环节会看识别性能，会对原理和实现代码部分做抽查提问，综合评定成绩。
  - 我们**鼓励做原创性探索**，即使性能不是很好，但有创新性、有价值、有意义的探索和尝试会有额外加分。
  - 原数完整据集下载地址：https://drive.google.com/file/d/180mSIiXN9RVDV2Xn1xcWNkMRm5J5MjN4/view?usp=sharing \
    或国内访问地址（4月17日前有效）：https://f.ws59.cn/f/du8xu130kba

## 1. 实验准备

In [8]:
## 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import torch
import os
device = "cuda" if torch.cuda.is_available() else "cpu"
# 可以根据需要导入其他库，比如librosa用于音频处理

## 2. 数据预处理(加载数据集)

In [9]:
## 请从如下地址下载数据集（4月17日前有效）：https://f.ws59.cn/f/du8yd2536vl
# 数据集基本信息如下
# 方言地区：DR1～DR8
# 性别：F/M
# 说话者ID：3大写字母+1阿拉伯数字
# 句子ID：句子类型（SA/SI/SX）+编号
# 详细介绍参见 https://blog.csdn.net/qq_39373179/article/details/103788208
#添加路径
# 上述链接下载的数据集已经
TrainDir = "Dataset/TRAIN"
TestDir = "Dataset/TEST"

## 请在这里写代码加载我们划分好的TIMIT训练集和测试集。或者原始完整版数据集。
from torch.utils.data import Dataset, DataLoader

class trainset(Dataset):
    def __init__(self,dir):
        self.dir_path=dir
        self.drs=os.listdir(self.dir_path)
        self.wavs=dict() # wav文件名：标注
        
        for dr in self.drs:
            speakers=os.listdir(os.path.join(self.dir_path,dr))
            for s in speakers:
                wavs=os.listdir(os.path.join(self.dir_path,dr,s))
                for w in wavs:
                    if w[-3:]=='wav': self.wavs[os.path.join(self.dir_path,dr,s,w)]=s[1:]
    def __getitem__(self, index):
        #第index的键和值
        wav=list(self.wavs.keys())[index]
        label=self.wavs[wav]
        dir_=wav.split('\\')[2]
        return wav,label,dir_
    def __len__(self):
        #return len(self.images)
        return len(self.wavs)

train_dataset=trainset(TrainDir)
validation_dataset=trainset(TestDir)
res=train_dataset[0]
res

('Dataset/TRAIN\\DR1\\FCJF0\\merge_result.wav', 'CJF0', 'FCJF0')

## 3. 特征提取

In [10]:
## 请编写或使用库函数提取MFCC等音频特征
# 提取特征
import librosa
import numpy as np

def extract_features(file_path, mfcc=True, chroma=True, mel=True):
    # Load audio file
    #print(file_path)
    y, sr = librosa.load(file_path, sr=None)

    features = []

    if mfcc:
        mfccs = librosa.feature.mfcc(y=y, sr=sr) 
        #print(f"@@@ mfccs shape:{mfccs.shape}")
        features.append(np.mean(mfccs, axis=1)) #20 * t
    if chroma:
        chroma = librosa.feature.chroma_stft(y=y, sr=sr)
        #print(f"@@@ chroma shape:{chroma.shape}")
        features.append(np.mean(chroma, axis=1)) #12 * t

    if mel:
        mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
        #print(f"@@@ mel_spectrogram shape:{mel_spectrogram.shape}")
        features.append(np.mean(mel_spectrogram, axis=1)) #128 *t
    
    return np.concatenate(features)


In [11]:
def evaluate_model(model, dataset):
    model.eval()  # 将模型设置为评估模式
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataset:
            inputs, labels, _ = data
            #print(inputs)
            inputs = torch.tensor([extract_features(file) for file in inputs]).to(device)  # 提取特征并转换为张量
            labels_=[]
            is_break=False
            for label in labels:
                if label in label_map:
                    labels_.append(label_map[label])
                else:
                    #print("@")
                    is_break=True
            if is_break: continue
            labels=labels_
            #labels = [label_map[label] for label in labels]  # 将人名转换为整数标签
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += batch_size
            #print(predicted,labels)
            labels=torch.tensor(labels).to(device)
            #print(sum(predicted==labels).item())
            print(labels) 
            print(predicted)
            correct+=sum(predicted==labels).item()
            
            #correct += sum(predicted == labels)

    accuracy = correct / total
    print('Accuracy on validation/test set: {:.2f}%'.format(100 * accuracy))
    return accuracy*100
#evaluate_model(model, validation_loader) 

In [12]:
len(validation_dataset)

924

## 4. 模型选择和训练

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# 定义DNN模型
class SpeakerRecognitionDNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SpeakerRecognitionDNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 512)
        self.fc2 = nn.Linear(512, 1024)
        self.fc3 = nn.Linear(1024, num_classes)
        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.softmax(x)
        return x

# 设置模型参数和超参数
input_size = 160  # 特征向量的大小
num_classes = len(set(train_dataset.wavs.values()))  # 类别数，根据数据集的说话人数量确定
learning_rate = 0.001
num_epochs = 100
batch_size = 32

# 初始化模型
model = SpeakerRecognitionDNN(input_size=input_size, num_classes=num_classes)

# 定义损失函数和优化器
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 准备数据加载器
# 创建人名到整数标签的映射
label_map = {label: idx for idx, label in enumerate(set(train_dataset.wavs.values()))}

# 准备数据加载器
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=True)

# 训练模型
model = model.to(device)


is_from_pth=False
if is_from_pth:
    path_="model_epoch_107.pth"
    model=load_model(model,path_ )
    t_e=int(path_.split('_')[-1].split('.')[0])
# 训练模型
for epoch in range(num_epochs):
    if is_from_pth:
        epoch_=epoch+t_e
    else: epoch_=epoch
    running_loss = 0.0
    #evaluate_model(model, validation_loader)
    model.train()
    for i, data in enumerate(train_loader, 0):
        inputs, labels, _ = data
        inputs = torch.tensor([extract_features(file) for file in inputs]).to(device)  # 提取特征并转换为张量
        labels = torch.tensor([label_map[label] for label in labels]).to(device)
          # 将人名转换为整数标签
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if (i+1) % 10 == 0:  # 每10个mini-batch打印一次loss
            print(f'Epoch [{epoch_+1}/{num_epochs+epoch_}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/10}')
            running_loss = 0.0
    #total += labels.size(0)
    # 在每轮训练之后进行评估
    print("Evaluation after epoch", epoch_ + 1)
    acc_= evaluate_model(model, validation_loader)  # 假设validation_dataset是验证集的数据加载器
    #写入log- acc
    with open('log.txt', 'a') as f:
        f.write(f"Epoch {epoch_+1}, Accuracy: {acc_} %\n")
    # 保存模型
    if (epoch_+1) %5==0:
        torch.save(model.state_dict(), f"model_epoch_{epoch_+1}.pth")
print('Finished Training')



KeyboardInterrupt: 

## 5. 评价指标(准确率Accuracy)

In [ ]:
## 请编写代码或使用库函数accuracy_score计算测试集上的准确率Accuracy
def load_model(model, model_path):
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model
test_=load_model(model, "model_epoch_100.pth")
evaluate_model(test_, validation_loader)

FileNotFoundError: [Errno 2] No such file or directory: 'model_epoch_100.pth'

##  6. 分析和可视化

In [ ]:
## 请使用matplotlib等可视化库对你的实验结果进行可视化分析。
## 包括但不限于准确率的对比、错误分类的分析、特征的影响等。


## 7. 结果讨论
讨论你的模型性能，尝试解释为什么某些模型比其他模型表现好，以及可能的改进方法。

## 8. 保存模型（可选）
如果需要，可以在这里添加代码保存你的模型。